## Datacube setup

In [ ]:
import datacube
import datetime

dc = datacube.Datacube(app='land valuation')

## Set common parameters for analysis

In [ ]:
date_range = (
    datetime.datetime(2013, 1, 1),
    datetime.datetime(2018, 1, 1))

# # extents for Sydney area
# bounding_box_x = (150, 151.37)
# bounding_box_y = (-34.36, -32.96)

# extents for area around Kensington (small)
bounding_box_x = (151.1735, 151.2752)
bounding_box_y = (-33.9399, -33.8741)

# GDA94 / NSW Lambert
crs = 'epsg:3308'
resolution = (-40, 40)

## Load land valuation dataset

In [ ]:
ds_land_valuation = dc.load(
    product='valuation_pa',
    x=bounding_box_x,
    y=bounding_box_y, 
    output_crs=crs,
    resolution=resolution)

## Plot land valuation

In [ ]:
%matplotlib inline
ds_land_valuation_plot = ds_land_valuation.isel(time=[0,1,2,3,4]).band1.plot(
    col='time', col_wrap=2, robust=True, aspect=ds_land_valuation.dims['x'] / ds_land_valuation.dims['y'],
    subplot_kws={'xticks': [], 'yticks': [], 'xlabel': '', 'yLabel': ''})
ds_land_valuation_plot.set_axis_labels('','')

## Load railway station proximity dataset

In [ ]:
ds_rail_prox = dc.load(
    product='rail',
    x=bounding_box_x,
    y=bounding_box_y, 
    output_crs=crs,
    resolution=resolution)

# drop time dimension
ds_rail_prox = ds_rail_prox.isel(time=0).drop('time')

%matplotlib inline
ds_rail_prox_plot = ds_rail_prox.band1.plot(
   robust=True, size=4, aspect=ds_rail_prox.dims['x'] / ds_rail_prox.dims['y'])

In [ ]:
import numpy as np

bins = [0, 1000, 2000, 3000, 4000]
labels = ["0 - 1000 m", "1000 - 2000 m", "2000 - 3000 m", "3000- 4000 m", "4000+ m"]

rail_prox_values = ds_rail_prox.band1.values
bins = [0, 1000, 2000, 3000, 4000]
rail_prox_values_bin = np.digitize(rail_prox_values, bins)

ds_rail_prox_bin = ds_rail_prox.copy()
ds_rail_prox_bin.band1.values = rail_prox_values_bin

ds_rail_prox_bin = ds_rail_prox_bin.rename({'band1': 'rail'})

%matplotlib inline
ds_rail_prox_bin.rail.plot(
   robust=True, size=4, aspect=ds_rail_prox_bin.dims['x'] / ds_rail_prox_bin.dims['y'])


In [ ]:
import xarray as xr
value_and_rail = xr.merge([ds_land_valuation, ds_rail_prox_bin])

value_and_rail_masked_1 = value_and_rail.where(value_and_rail['rail'] == 1)
value_and_rail_masked_2 = value_and_rail.where(value_and_rail['rail'] == 2)
value_and_rail_masked_3 = value_and_rail.where(value_and_rail['rail'] == 3)
value_and_rail_masked_4 = value_and_rail.where(value_and_rail['rail'] == 4)
value_and_rail_masked_5 = value_and_rail.where(value_and_rail['rail'] == 5)

value_and_rail_masked_1 = value_and_rail_masked_1.assign_coords(distance_from_rail_station=('d < 1000m'))
value_and_rail_masked_2 = value_and_rail_masked_2.assign_coords(distance_from_rail_station=('1000m <= d < 2000m'))
value_and_rail_masked_3 = value_and_rail_masked_3.assign_coords(distance_from_rail_station=('2000m <= d < 3000m'))
value_and_rail_masked_4 = value_and_rail_masked_4.assign_coords(distance_from_rail_station=('3000m <= d < 4000m'))
value_and_rail_masked_5 = value_and_rail_masked_5.assign_coords(distance_from_rail_station=('4000m <= d'))

value_and_rail_all = xr.concat([value_and_rail_masked_1, value_and_rail_masked_2, value_and_rail_masked_3, value_and_rail_masked_4, value_and_rail_masked_5], dim='distance_from_rail_station')

value_and_rail_all_mean = value_and_rail_all.mean(dim=['x', 'y'], skipna=True)
value_and_rail_all_mean_pc = value_and_rail_all_mean / value_and_rail_all_mean.isel(time=0) * 100.0


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

plt.ylabel('Mean land value increase (% since 2013)')
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

value_and_rail_all_mean_pc.band1.attrs['units'] = '%'
value_and_rail_all_mean_pc.band1.plot(hue='distance_from_rail_station')
